In [3]:
import imageio
import os
import sys

import numpy as np
import skimage.transform

In [8]:
# NOTE. 1. load_colmap_data
import colmap_read_model as read_model

def load_colmap_data(realdir):

    camerasfile = os.path.join(realdir, "sparse/0/cameras.bin")
    camdata = read_model.read_cameras_binary(camerasfile)

    list_of_keys = list(camdata.keys())
    cam = camdata[list_of_keys[0]]
    print(f"[DEBUG] in `load_colmap_data`; #cameras = {len(cam)}")

    h, w, f = cam.height, cam.width, cam.params[0]
    hwf = np.array([h, w, f]).reshape([3, 1])

    imagesfile = os.path.join(realdir, "sparse/0/images.bin")
    imdata = read_model.read_images_binary(imagesfile)

    w2c_mats = []
    bottom = np.array([0, 0, 0, 1.]).reshape([1, 4])
    
    names = [imdata[k].name for k in imdata]
    print(f"[DEBUG] in `load_colmap_data`; #images = {len(names)}")
    perm = np.argsort(names)
    for k in imdata:
        im = imdata[k]
        R = im.qvec2rotmat()
        t = im.tvec.reshape([3, 1])
        
        m = np.concatenate([np.concatenate([R, t], axis=1), bottom], axis=0)
        w2c_mats.append(m)

    w2c_mats = np.stack(w2c_mats, axis=0)
    c2w_mats = np.linalg.inv(w2c_mats)

    poses = c2w_mats[:, :3, :4].transpose([1, 2, 0])
    poses = np.concatenate([
        poses, 
        np.tile(
            hwf[..., np.newaxis],
            [1, 1, poses.shape[-1]] 
        )
    ], axis=1)

    points3dfile = os.path.join(realdir, "sparse/0/points3d.bin")
    pts3d = read_model.read_points3d_binary(points3dfile)

    # NOTE: must switch to [-u, r, -t] from [r, -u, t], NOT [r, u, -t]
    poses = np.concatenate([
        poses[:, 1:2, :], 
        poses[:, 0:1, :], 
        -poses[:, 2:3, :], 
        poses[:, 3:4, :], 
        poses[:, 4:5, :]
    ], axis=1)

    return poses, pts3d, perm


In [9]:
# 2. NOTE: save_poses

def save_poses(basedir, poses, pts3d, perm):
    pts_arr = []
    vis_arr = []
    for k in pts3d:
        pts_arr.append(pts3d[k].xyz)
        cams = [0] * poses.shape[-1]

        for ind in pts3d[k].image_ids:
            if len(cams) < ind - 1:
                raise ValueError(f"[ERROR] the correct camera poses for current points cannot be accessed!")
            
            cams[ind-1] = 1
        vis_arr.append(cams)
    
    pts_arr = np.array(pts_arr)
    vis_arr = np.array(vis_arr)
    print(f"[DEBUG] in `save_poses`; points={pts_arr.shape}, visibility={vis_arr.shape}")

    zvals = np.sum(
        -(pts_arr[:, np.newaxis, :].transpose([2, 0, 1]) - poses[:3, 3:4, :]) 
        * poses[:3, 2:3, :]
    , axis=0)
    valid_z = zvals[vis_arr==1]
    print(f"[DEBUG] depth stats: {valid_z.min()}, {valid_z.max()}, {valid_z.mean()}")


    save_arr = []
    for i in perm:
        vis = vis_arr[:, i]
        zs = zvals[:, i]
        zs = zs[vis==1]

        # NOTE: near & far bound
        close_depth, inf_depth = np.percentile(zs, 0.1), np.percentile(zs, 99.9)

        save_arr.append(np.concatenate([
            poses[..., i].ravel(), 
            np.array([close_depth, inf_depth])
        ], axis=0))
    save_arr = np.array(save_arr)

    np.save(os.path.join(basedir, "poses_bounds.npy"), save_arr)
    
    return

In [10]:
# NOTE: 3. gen_poses

def gen_poses(basedir, match_type, factors=None):

    files_needed = [
        f"{f}.bin" for f in ["cameras", "images", "points3D"]
    ]
    if os.path.exists(os.path.join(basedir, "sparse/0")):
        files_had = os.listdir(os.path.join(basedir, "sparse/0"))
    else:
        files_had = []

    if not all([f in files_had for f in files_needed]):
        
        raise ValueError(f"[ERROR] some required files missing! run COLMAP.")
    else:
        print(f"[DEBUG] no need to run COLMAP.")

    print(f"[DEBUG] post-COLMAP:")

    poses, pts3d, perm = load_colmap_data(basedir)

    save_poses(basedir, poses, pts3d, perm)

    if factors is not None:
        print(f"[DEBUG] {factors=}")
        # TODO: `minify here`

    print(f"[DEBUG] done with img2poses.")

    return True

In [11]:
# NOTE: let's generate!

basedir = "/mnt/d/nerf/colmap/"

gen_poses(basedir, None, None)

[DEBUG] no need to run COLMAP.
[DEBUG] post-COLMAP:
[DEBUG] in `load_colmap_data`; #cameras = 5
[DEBUG] in `load_colmap_data`; #images = 17
[DEBUG] in `save_poses`; points=(23027, 3), visibility=(23027, 17)
[DEBUG] depth stats: 1.3743545915451887, 212.8680066091597, 20.677908003518365
[DEBUG] done with img2poses.


True